Long time ago, Fama and MacBeth proposed a two-step Least Squares approach to estimate the amount of systematic risk (beta) and its market price of risk (lambda) in two consecutive steps.

$$
\quad E[ r_{i,t}] - r_t = \alpha + \lambda^{f} \times \beta_i^{f}   
$$

where $\lambda^{f}$ is the unconditional market price of $f$-risk.

To estimate that **unconditional market price of risk**, Fama and MacBeth propose a two stage estimation procedure: First, regress the returns onto the risk factors to get betas of assets (i.e. time-series regressions). Second, regress asset returns in each time point (cross-sectional regression) onto the betas from stage 1. 


We use a fama-macBeth regressions to identify $\lambda$ and $\beta$ using a two step approach

**Step 1:** time-series regressions for each individual stock returns r to recover the stock's  $\beta$
$$
r_{t=1,...,T} - r_{f} = \alpha + \beta (r_{MKT,t}-r_{f}) + \epsilon_{t=1,...,T}
$$
$$
\\
$$

**Step 2:** cross-sectional regressions for each time point t to recover $\lambda_f$
$$
r_{i=1,...,377} - r_f = \alpha + \lambda \times \beta_{i=1,...,377} + \epsilon_{i=1,...,377}
$$

**Step 3:**
$$
\lambda_{FMacB} = \frac{1}{T} \sum_t \lambda_t, \qquad \text{and} \qquad t(\lambda_{FMacB}) = \frac{\lambda_{FMacB}}{std(\lambda_t)/ \sqrt{T}}
$$

0.636015306122449

# Class: FMacBeth for estimating UNCONDITIONAL BETA AND MARKET PRICES OF RISK for FF-7 Model

In [1]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import copy
from scipy import stats


In [ ]:
class FMacBeth():
    def __init__(self, r, f,rf):
        #input: r: return panel, pandas
        #input: f: factor panel, pandas
        #input: rf: risk-free rate, pandas
        self.r = copy.copy(r)
        self.f = copy.copy(f)
        self.rf = copy.copy(rf)
    
     
        #BETA
        self.beta = r.agg(['mean']).T
        self.beta["beta_MKT"] = 0
        self.beta['beta_SMB'] = 0
        self.beta['beta_HML'] = 0
        self.beta['beta_RMW'] = 0
        self.beta['beta_CMA'] = 0
        self.beta['beta_Rev'] = 0
        self.beta['beta_Mom'] = 0
        
        self.beta.drop(['mean'], axis=1, inplace=True) 
       
        #BETA_R2 (i.e. R2 of linear factor model regression)
        self.betaR2 = r.agg(['mean']).T
        self.betaR2.rename(columns={'mean': 'beta_R2'}, inplace=True)  
        
  
        
    def TSRegression_step1(self):
        for i in range(0,self.r.shape[1]): 
            X_ = self.f
            X_ = sm.add_constant(X_)
            gls_ = sm.GLS(self.r.iloc[:,i]-self.rf.iloc[:,0], X_).fit()
            betas_i = gls_.params.values 
            for j in range(1,self.f.shape[1]+1):
                self.beta.iloc[i,j-1] = betas_i[j]
             
            self.betaR2.iloc[i,0] = gls_.rsquared_adj
                
           
    def CSRegression_step2(self):
        lambdas = np.zeros((self.r.shape[0],self.f.shape[1]))     
        lambdasR2 = np.zeros((self.r.shape[0], 1))
        
        X_ = self.beta
        X_ = sm.add_constant(X_)
        
        for t in range(0,self.r.shape[0]):
            y_t = self.r.iloc[t,:] - self.rf.iloc[t,0]
            gls_t = sm.GLS(y_t, X_).fit()
            
            lambdasR2[t,0] = gls_t.rsquared_adj 
            lambdas[t,:] = gls_t.params.values[1:]
            
             
             
            
        
        lambda_FB = np.mean(lambdas, axis=0) / self.r.shape[0]
        self.lambdas = pd.DataFrame(data=lambdas, index=self.r.index)
        
        self.lambdas_FB = pd.DataFrame(data=lambda_FB, index = ['MKT', 'SMB', 'HML', 'RMW', 'CMA', 'Rev', 'Mom'])
        
        self.t_lambdas_FB = self.lambdas_FB / ( self.lambdas.std(axis=0) / np.sqrt(self.r.shape[0]) )
        
        self.lambdasR2 = pd.DataFrame(data=lambdasR2, index=self.r.index) 
        
    
        
        
        
                
             
                
        